In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow import keras

print(tf.version.VERSION)

2022-06-18 22:45:01.219187: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-18 22:45:01.219212: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.9.1


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10),
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


2022-06-18 22:45:02.601771: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-18 22:45:02.601798: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-18 22:45:02.601817: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (toor): /proc/driver/nvidia/version does not exist
2022-06-18 22:45:02.602049: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [5]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)
tfjs.converters.save_keras_model(model, 'tfmnist_json')

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.1261 - sparse_categorical_accuracy: 0.6890
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4179 - sparse_categorical_accuracy: 0.8880
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2904 - sparse_categorical_accuracy: 0.9190
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2007 - sparse_categorical_accuracy: 0.9510
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1524 - sparse_categorical_accuracy: 0.9680


In [6]:
def process_path(file_path):
    img = tf.io.read_file(file_path)
    img = tf.io.decode_png(img, channels=1)
    return tf.image.resize(img, [28, 28])

In [7]:
import numpy as np

In [8]:
images = [process_path(img) for img in ('images/8.png', 'images/3.png', 'images/2.png')]

In [9]:
predictions = model.predict(tf.reshape(images, (-1, 28 * 28)) / 255.0)

1/1 [==============================] - 0s 62ms/step


In [10]:
pd.DataFrame(predictions)

,0,1,2,3,4,5,6,7,8,9
0,-1.495842,-0.739649,0.049677,1.133056,-1.877375,0.372429,-0.785565,-1.184312,1.188074,-0.783215
1,-2.071520,-0.025058,0.019247,1.009438,-2.006342,-0.507646,0.135799,1.463819,-1.313075,0.010183
2,-0.544475,0.270408,2.496238,0.016574,-1.834643,-0.985348,-0.056792,-0.697887,-1.672723,-0.801232
